In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import requests, censusdata, zipfile
import re
import os
import sys
import urllib, json, requests
from os import path
from pathlib import Path
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

## 1. County-Level COVID-19 Case Data

In [ ]:

raw_dir = "../raw_data/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/"
start_date_file = "03-23-2020.csv"

In [ ]:
# Create DataFrame of confirmed cases by county and date

data_list = []

for file in os.listdir(raw_dir):
    if file[-4:] == ".csv" and file >= start_date_file:
        entry = pd.read_csv(raw_dir + file)
        entry = entry.loc[(entry.Country_Region == "US") & (entry.FIPS.apply(str).str.len() == 7), ["FIPS", "Confirmed"]].astype(int)
        entry = entry.set_index("FIPS").rename(columns = {"Confirmed": file[:-4]})
        entry = entry[~entry.index.duplicated()]
        data_list.append(entry)
        
confirmed = pd.concat(data_list, axis = 1)

In [ ]:
# Create DataFrame of new confirmed cases

new_confirmed = confirmed.diff(axis = 1)
new_confirmed[start_date_file[:-4]] = confirmed[start_date_file[:-4]]

In [ ]:
new_confirmed.to_csv("../processed_data/new_confirmed.csv")

## 2. US county demographic data EDA

#### 2.1 Pull downloaded Kaggle data (requires auth) 
Basic county-level demographic data from JHU and US Census was downloaded from Kaggle to a CSV since download requires username / password authentication. The data contains the unique FIPS code, county / state names, male and female populations, median county age, as well as the latitude and longitude of the center of the county.

In [ ]:
kaggle_demos = pd.read_csv(raw_dir/'us_county.csv')
kaggle_demos.head()

#### 2.2 Request IHME health data

Some additional health data was downloaded from the Institute for Health Metrics and Evaluation (IHME). The data contains time series records of mortality risk and life expectancy measures across every county for several age ranges dating back to the 1980s. We cleaned and standardized this data to only include the most recent sample of county average life expectancy at birth and mortality risk for the 65 to 85 year old population (the most vulnerable to the effects of COVID-19). The data also includes a FIPS code.

In [ ]:
if 'ihme_health.csv' not in [_.name for _ in raw_dir.glob('*')]:
    h_url = 'http://ghdx.healthdata.org/sites/default/files/record-attached-files/'
    h_file = 'IHME_USA_COUNTY_LE_MORTALITY_RISK_1980_2014_NATIONAL_STATES_DC_CSV.zip'
    r = requests.get(h_url+h_file, stream=True)
    with open(raw_dir/'ihme_health.zip', 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
    with zipfile.ZipFile(raw_dir/'ihme_health.zip', 'r') as zip_ref:
        zip_ref.extractall(raw_dir/'imhe_health/')
    health_pth = Path(raw_dir/'imhe_health/')
    assert health_pth.exists(), 'no health files found'
    files = health_pth.glob('*.csv')
    
    health_dfs = []
    for f in files:
        try:
            df = pd.read_csv(f)
            max_year = df.year_id.max()
            # logic to get single rows for pivot table
            df = df[(df.year_id == max_year) & (df.sex == "Both") \
                    & ((df.age_name == "0") | (df.age_name == "65 to 85"))]
            pvt = pd.pivot_table(df, values = 'val', columns = 'measure_name',
                    index=['FIPS', 'location_name']).reset_index()
            health_dfs.append(pvt)
        except Exception as e:
            print(e); continue
            
    health = pd.concat([_ for _ in health_dfs if _.shape[0] > 1])
    health.to_csv(raw_dir/'ihme_health.csv')
else:
    health = pd.read_csv(raw_dir/'ihme_health.csv')

#### 2.3 Census income data

We downloaded county-level income data from the most recent Census survey (2018) and processed the data to find the estimated number of impoverished people in each county, as well as the median household income. The data again includes a FIPS code.

In [ ]:
r = requests.get('https://www2.census.gov/programs-surveys/saipe/datasets/2016/2016-state-and-county/est16all.txt')
t = r.text

lines = t.split('\n')
income_dict = {}
for l in lines:
    name = l[193:238].strip()
    if name == '': continue
    fin_demos = {
        'fips_state': l[0:2].strip(),
        'fips_county': l[3:6].strip(),
        'all_poverty': l[7:15].strip(),
        'median_hh_income': l[133:139].strip()
    }
    income_dict[name] = fin_demos

income = pd.DataFrame(income_dict).T
def fix_fips(s):
    if len(s) == 3: return s
    elif len(s) == 2: return f'0{s}'
    else: return f'00{s}'
    
# fix erros in Kalawao county
income = income.replace('.', np.nan)
income['all_poverty'] = income.all_poverty.astype(float)
income['median_hh_income'] = income.median_hh_income.astype(float)

income['fips_county_pad'] = income.fips_county.map(fix_fips)
income['fips'] = (income.fips_state + income.fips_county_pad).astype(int)
income.head()

#### 2.4 Land area

Finally, we pull census data for the land area (in square miles) of each county as of 2010, which is the most recent year available. The data includes the FIPS code.

In [ ]:
landvar = 'LND110210D'
land = pd.read_csv(raw_dir/'LND01.csv')[['STCOU', landvar]].rename(
    columns={'STCOU':'fips', landvar:'sq_miles'})
land.head()

#### 2.5 Merge

We merge all of these datasets on FIPS code.

In [ ]:
demos = pd.merge(
    kaggle_demos, health.set_index('FIPS')[['Life expectancy', 'Mortality risk']],
    left_on='fips', right_index=True).merge(
        income.set_index('fips')[['all_poverty', 'median_hh_income']],
        left_on='fips', right_index=True).merge(
            land.set_index('fips'), left_on='fips', right_index=True)

def fix_colname(s): return s.lower().strip().replace(' ', '_')
demos.columns = [fix_colname(_) for _ in demos.columns]
demos.head()

#### 2.6 Enrich

We engineer two features that we intuitively believe may have some relationship to the spread of COVID-19. The first is related to the rate at which the population is impoverished. We have estimates from Census of the total number of inhabitants living below the poverty line, but here we standardize by the population of the county to arrive at an estimate of the percent of the county that is impoverished. We also use the population variable and the estimated land area variable to determine the population density in terms of inhabitants per square mile.

In [ ]:
demos['pct_impoverished'] = demos.all_poverty.div(demos.population)
demos['pop_density'] = demos.population.div(demos.sq_miles)

In [ ]:
demos.to_csv(processed_dir/'demographic_data.csv', index=False)

#### 2.7 Data Understanding
We'll visualize some key features to get a better understanding of the demographic data.

In [ ]:
features = 'median_age female_percentage life_expectancy mortality_risk \
median_hh_income pct_impoverished pop_density'.split()

In [ ]:
cm = sns.clustermap(demos[features].corr(method='spearman'), annot=True, fmt='.1f', linewidths=0.5,
              mask=np.eye(len(features)), cmap='RdYlGn', dendrogram_ratio=.1,
              figsize=(6,6), cbar=False)
cm.fig.subplots_adjust(top=.9)
cm.cax.set_visible(False)
cm.fig.suptitle('Hierarchical cluster of key demographic features')
plt.show()

In [ ]:
demos.pct_impoverished.hist()
plt.title('Histogram of population poverty rate by county')
plt.show()

In [ ]:
demos.plot.scatter('pct_impoverished', 'mortality_risk', alpha=0.3)
plt.xlabel('Percent of county living in poverty')
plt.ylabel('Estimated mortality risk')
plt.title('Relationship between poverty and mortality risk')

X = demos.dropna()
lm = LinearRegression().fit(X.pct_impoverished.values.reshape(-1,1), X.mortality_risk)
xs = np.linspace(0, 0.5)
ys = lm.predict(xs.reshape(-1,1))
r2 = lm.score(X.pct_impoverished.values.reshape(-1,1), X.mortality_risk)
print(f"R2 score of poverty and mortality risk is {r2:.3f}")
plt.plot(xs,ys, color='black')
plt.legend([f'$R^2 = {r2:.2f}$', 'Samples'])
plt.show()

In [ ]:
demos.plot.scatter('pop_density', 'median_hh_income', alpha=0.3)
plt.xscale('log'); plt.yscale('log')
plt.xlabel('Log population density')
plt.xlabel('Log median household income')
plt.title('Relationship between poverty and mortality risk'); plt.show()

## 3. Protest Data

#### 3.1 Data Cleaning

In [ ]:
protest_data = pd.read_csv("../raw_data/protest_AECLD.csv")

In [ ]:
empty_search = r'no\ report'
empty = protest_data.NOTES.str.findall(empty_search)

count_empty = 0
for i in empty:
    if i:
        count_empty = count_empty + 1

count_empty

In [ ]:
sr = r'\[size=(.*)\]'
nums = r'\d+|few\ dozen*|few\ hundred*|few\ thousand*|several\ dozen*|several\ hundred*|several\ thousand*|dozen*|hundred*|thousand*'
sizes = protest_data.NOTES.str.findall(sr)
unique_sizes = [list(x) for x in set(tuple(x) for x in sizes)]
size_temp_df = pd.DataFrame([''.join(x) for x in sizes], columns = ['string'])
size_labs = size_temp_df.string.str.findall(nums)

count_recovered = 0
for i in size_labs:
    if i:
        count_recovered = count_recovered + 1

count_recovered

print(f'{count_recovered} of the {len(protest_data) - count_empty} non-emtpy records were retrieved.')

size_cleaned = [None] * len(protest_data)

dozen = 12
few_dozen = 50
hundred = 100
few_hundred = 500
thousand = 1000
few_thousand = 5000

for ii in range(len(size_labs)):
    
    vals = size_labs[ii]
    
    if any(st == "few dozen" for st in vals) | any(st == "several dozen" for st in vals):
        size_cleaned[ii] = few_dozen
    elif any(st == "few hundred" for st in vals) | any(st == "several hundred" for st in vals):
        size_cleaned[ii] = few_hundred
    elif any(st == "few thousand" for st in vals) | any(st == "several thousand" for st in vals):
        size_cleaned[ii] = few_thousand
    elif any(st == "dozen" for st in vals):
        size_cleaned[ii] = dozen
    elif any(st == "hundred" for st in vals):
        size_cleaned[ii] = hundred
    elif any(st == "thousand" for st in vals):
        size_cleaned[ii] = thousand
    else:
        size_cleaned[ii] = np.mean(np.array(vals, dtype=np.float32).astype(np.float))

print(len(size_cleaned))
protest_data["size"] = size_cleaned


In [ ]:
protest_data["99 or less"] = (protest_data["size"] <= 99) * 1
protest_data["100 to 499"] = ((protest_data["size"] > 99) &(protest_data["size"] <= 499)) * 1
protest_data["500 to 999"] = ((protest_data["size"] > 499) & (protest_data["size"] <= 999)) * 1
protest_data["1000 to 4999"] = ((protest_data["size"] > 999) & (protest_data["size"] <= 4999)) * 1
protest_data["more than 4999"] = ((protest_data["size"] > 4999)) * 1
protest_data

sizes = ["99 or less", "100 to 499", "500 to 999", "1000 to 4999", "more than 4999"]

In [ ]:
convert={'fips': lambda x: str(x), 'EVENT_DATE': lambda x: pd.to_datetime(x)}
protest_fips = pd.read_csv("../processed_data/protests_fips.csv", converters = convert)
protest_fips

In [ ]:
protest_data.EVENT_DATE = pd.to_datetime(protest_data.EVENT_DATE)
protest_data = protest_data.replace('', np.nan)
protest_fips["EVENT_DATE"] = pd.to_datetime(protest_fips.EVENT_DATE)
protest_data_full = protest_data.join(protest_fips.fips,  how = "left")
var_list = ["fips", "EVENT_DATE","99 or less", "100 to 499", "500 to 999", "1000 to 4999", "more than 4999"]
protest_data_full
protest_data_clean = protest_data_full[var_list]
protest_data_clean["unknown"] = (protest_data_clean[["99 or less", "100 to 499","500 to 999", "1000 to 4999", "more than 4999"]].sum(axis = 1) == 0).astype(int)
protest_data_clean.dtypes

In [ ]:
protest_data_clean.to_csv("../processed_data/protests.csv", index = False)

#### 3.2 FIPS Codes

In [ ]:
fips_codes = []

for i in range(len(protest_data)):
    
    temp_url = "https://geo.fcc.gov/api/census/area?lat=" +str(protest_data.LATITUDE[i]) +"&lon=" + str(protest_data.LONGITUDE[i])+ "&format=json"
    
    with urllib.request.urlopen(temp_url) as url:
        data = json.loads(url.read().decode())
    
    if not data["results"]:
        fips_codes.append("")
    else:
        fips_codes.append(data["results"][0]["county_fips"])

In [ ]:
protest_data["fips_county"] = fips_codes
protest_data["fips_county"] = protest_data.fips_county.astype(str)

#### 3.3 Visualization and EDA

In [ ]:
convert={'fips': lambda x: str(x), 'EVENT_DATE': lambda x: pd.to_datetime(x)}
protest_df = pd.read_csv("../processed_data/protests.csv", converters = convert)
protest_df

In [ ]:
## Data cleaning for plotting
## exclude entires from before George Floyd's death
protest_df = protest_df[protest_df.EVENT_DATE > "2020-05-24"]

## collapse dummy variables to categories
sizes = ["99 or less", "100 to 499", "500 to 999", "1000 to 4999", "more than 4999", "unknown"]
protest_df["size"] = protest_df[sizes].idxmax(axis = 1)
protest_df

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))
ax = sns.countplot(x="size", 
                   data=protest_df[protest_df["size"] != "unknown"],
                  order = ["99 or less", "100 to 499", "500 to 999", "1000 to 4999", "more than 4999"])
ax.set_title("Counts of Protest Sizes", fontsize = "15")
ax.set_xlabel("Protest Size")
ax.set_ylabel("Count")

In [ ]:
protest_df["size"].value_counts()

In [ ]:
demo_df = pd.read_csv("../processed_data/demographic_data.csv")
demo_df.fips = [str(item).zfill(5) for item in demo_df.fips]
protest_fips = protest_df.fips.unique()

protest_fips_counts = protest_df.fips.value_counts().reset_index()
protest_fips_counts.columns = ["fips", "num"]
protest_fips_counts.sort_values(by = ["num"], inplace = True)

print(f"Of the total {len(demo_df)} FIPS code counties, {len(protest_fips)} had protests and {len(demo_df) - len(protest_fips)}.")

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.hist(protest_fips_counts["num"], 75)
ax.set_xlabel("Number of Protests")
ax.set_ylabel("Number of Counties")
ax.set_title("Distribution of Numbers of Protests", fontsize = "15")

In [ ]:
demo_df = demo_df.merge(protest_fips_counts, on = ["fips"], how = "left")
demo_df = demo_df.rename(columns = {"num":"num_protests"})
demo_df["num_protests"] = demo_df["num_protests"].fillna(0).astype(int)

In [ ]:
values = [0, 1, 2]
conditions = [
    (demo_df["num_protests"]==0),
    (demo_df["num_protests"] > 0) & (demo_df["num_protests"] < 6),
    (demo_df["num_protests"] > 5)
]

demo_df["protests"] = np.select(conditions, values)

In [ ]:
plot_var = ["median_age", "female_percentage", "life_expectancy","pct_impoverished"]

fig, axes = plt.subplots(2,2, figsize = (12,8))

for i, ax in enumerate(axes.ravel()):
    demo_var = plot_var[i]
    sns.kdeplot(data = demo_df[demo_df["protests"] == 0][demo_var], ax = ax, label = "No Protests")
    sns.kdeplot(data = demo_df[demo_df["protests"] == 1][demo_var], ax = ax, label = "5 or Fewer")
    sns.kdeplot(data = demo_df[demo_df["protests"] == 2][demo_var], ax = ax, label = "6 or More")
    ax.set_xlabel(demo_var)
    ax.set_ylabel("Density")
    ax.set_title(demo_var)
fig.suptitle("Distributions of Demographic Data by Number of Protests", y = 0.95, fontsize = "15")
fig.subplots_adjust(hspace = 0.5)

## 4. Model

#### 4.1 Creating the Prediction Dataset

We aggregate the processed COVID-19 data with the demographic data, and create new rows in an augmented dataset including 14-day windows of COVID-19 cases from mid-March as distinct examples.

In [ ]:
covid_df = pd.read_csv("../processed_data/new_confirmed.csv")
demographics_df = pd.read_csv("../processed_data/demographic_data.csv")

df = covid_df.merge(demographics_df, left_on="FIPS", right_on="fips")
df.head()

In [ ]:
WINDOW_SIZE = 15

date_cols = df.columns[1:np.where(df.columns == 'fips')[0][0]]
nondate_cols = df.columns[np.where(df.columns == 'fips')[0][0]:]

augmented_data = []
augmented_index = []
    
for index, row in tqdm(df.iterrows()):
    for i, col in enumerate(date_cols[:-WINDOW_SIZE]):
        series = row[date_cols[i:i+WINDOW_SIZE]].reset_index(drop=True)
        series_dict = {f"{14 - k}_before": v for k, v in series.to_dict().items()}
        series_dict.update(row[nondate_cols].to_dict())
        augmented_data.append(series_dict)
        augmented_index.append(f"{row['fips']}_{date_cols[i+WINDOW_SIZE-1]}")

augmented_df = pd.DataFrame(data=augmented_data, index=augmented_index)
augmented_df = augmented_df[1:]
augmented_df.to_csv("../processed_data/combined.csv")

#### 4.2. Preprocess Data

- We decided to remove rows with null values (given the high abundance of training examples) and to remove rows which included a stray value one day in New York as per https://github.com/CSSEGISandData/COVID-19/issues/3103.
- We assigned the 14-day covid history as separate predictors, as well as a few demographic indicators
- We standardized the data given high variance between the ranges of different features

In [ ]:
def num_to_date(i):
    if i>9:
        return str(i)
    return "0" + str(i)

augmented_df = pd.read_csv("../processed_data/combined.csv", index_col=0)

# drop null values
augmented_df = augmented_df.dropna(subset=([f"{k}_before" for k in range(15)]+['pct_impoverished']))

# remove data with spike from NY county https://github.com/CSSEGISandData/COVID-19/issues/3103
augmented_df = augmented_df.drop(["36061_08-31-2020"]+[f"36061_09-{num_to_date(i)}-2020" for i in range(1,15)])

In [ ]:
predictors = [f"{k + 1}_before" for k in range(14)] + ['median_age', 'female_percentage', 'life_expectancy', 'pct_impoverished', 'median_hh_income']
X = augmented_df[predictors]
y = augmented_df['0_before']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=209)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### 4.3 Prediction

- We fit a model to the scaled, cleaned data and report MSE and coefficient values

In [ ]:
baseline = LinearRegression()
baseline.fit(X_train_scaled, y_train)

y_train_predict = baseline.predict(X_train_scaled)
y_test_predict = baseline.predict(X_test_scaled)

train_mse = mean_squared_error(y_train_predict, y_train)
test_mse = mean_squared_error(y_test_predict, y_test)

print(f"Train MSE: {train_mse}\n Test MSE: {test_mse}\n")

print("Coefficient values")
for coef, col in zip(baseline.coef_, predictors):
    print(f"{col}: {coef}")